In [ ]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Output, Input
import ccxt
import pandas as pd
import plotly.graph_objs as go

import webbrowser

In [ ]:
# Set up Binance API
binance = ccxt.binance()
symbol = 'BTC/USDT'

In [ ]:
import import_ipynb
from predictor import XGBoostPredictor, RNNPredictor, LSTMPredictor
# model = XGBoostPredictor('../models/xgboost_model.pkl')
# model = RNNPredictor('../models/rnn_model.h5')

from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler(feature_range=(0,1))

import numpy as np

def predict(new_data, model_name):
    if model_name == 'RNN':
        model = RNNPredictor('../models/rnn_model.h5')  
    elif model_name == 'LSTM':
        model = LSTMPredictor('../models/lstm_model.h5')
    elif model_name == 'XGBoost':
        model = XGBoostPredictor('../models/xgboost_model.pkl')
    
    new_data.drop("timestamp",axis=1,inplace=True)
    scaler.fit_transform(new_data.values)
    
    inputs=new_data[len(new_data)-100:].values
    inputs=inputs.reshape(-1,1)
    inputs=scaler.transform(inputs)

    X_test=np.array(inputs)

    X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
    closing_price=model.predict(X_test)
    closing_price=scaler.inverse_transform(closing_price)

    return closing_price

In [ ]:
# Initialize Dash app
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(id='model-name',options=[
                    {'label': 'LSTM', 'value': 'LSTM'},
                    {'label': 'RNN', 'value': 'RNN'},
                    {'label': 'XGBoost', 'value': 'XGBoost'},
                     ],
                    value='LSTM'
                 ),
    dcc.Graph(id='live-graph'), 
    html.Div(id='remaining-time'),
    dcc.Interval(   
        id='interval-component',
        interval=15 * 1000,  # in milliseconds
        n_intervals=0
    ),
])

In [ ]:
@app.callback([Output('live-graph', 'figure'), Output('remaining-time', 'children')],
              [Input('interval-component', 'n_intervals'), Input('model-name', 'value')])
def update_graph(n, model_name):
    ohlcv = binance.fetch_ohlcv(symbol, '1h', limit=1000)
    df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    
    input_data =  df[['timestamp','close']]

    predictions = predict(input_data[['timestamp', 'close']], model_name)
    predictions = list(np.concatenate(predictions))

    trace = go.Candlestick(
        x=df['timestamp'],
        open=df['open'],
        high=df['high'],
        low=df['low'],
        close=df['close'],
        name=symbol
    )
    
    trace_predictions = go.Scatter(
        x=df['timestamp'].iloc[-100:],
        y=predictions,
        mode='lines',
        name='Predicted Close Prices',
        line=dict(color='orange')
    )
    
    layout = go.Layout(
        title=f'Live OHLCV Chart - {symbol}',
        xaxis=dict(title='Time'),
        yaxis=dict(title='Price'),
        showlegend=True,
        height=700
    )
    
    remaining_time = 10 - (n % 10)  # Calculate remaining seconds
    
    return {'data': [trace, trace_predictions], 'layout': layout}, f'model: {model.model_filename}, input: {model_name}'


# Start the app.

In [ ]:
if __name__ == '__main__':
    webbrowser.open('http://127.0.0.1:8050/', new=0, autoraise=True)
    app.run_server(debug=False)